In [17]:
import numpy as np
from classy import Class
from Francisco import trapezoid, right_rectangle, factors, xy_values
import scipy.interpolate as sp


In [2]:
M_sol = 1.989e30 # solar mass in kg
Mpc = 3.086e22 # Mpc in m
c=np.logspace(-11.3,-5, 40)
kvec = np.logspace(-4,np.log10(100),100)
measured_omegacdm=.1188
Sig_8 = 0.811
A_s = 2.1e-9

In [3]:
commonsettings = {
          'N_ncdm':1,
          'use_ncdm_psd_files': 1,
          'm_ncdm': 7100,
          'T_ncdm':0.7,
          'output':'mPk',
          'P_k_max_1/Mpc':100.0,
          # The next line should be uncommented fgor higher precision (but significantly slower running)
          'ncdm_fluid_approximation':3,
          # You may uncomment this line to get more info on the ncdm sector from Class:
          'background_verbose':1,
          'Maximum q':100
         } 

In [4]:
dat = np.load('1x0.00049x3e-09-data.npz')
f = dat['f_full']

print(dat.files)
omega_h_h = dat['omega_s']
a,b,c,d,e = xy_values('1x0.00049x3e-09-data.npz', 20)
np.savetxt("Spec", np.column_stack((a,b)))
spec_file  = 'Spec'

['eps', 'f', 'eps_full', 'f_full', 'omega_s', 'a', 'T', 'Lep']


In [5]:
def ideal_sigma8 (spec_file, omega_h_h):
    
    NH = Class()
        #use method .set() to 
    NH.set(commonsettings)
    othersettings = { 'ncdm_psd_filenames': spec_file,
                      'omega_cdm': measured_omegacdm - omega_h_h, 
                      'omega_ncdm': omega_h_h, 
                      'A_s': 2.1e-9,
                    }

    #use method .compute() to get data for my specific 'Spec-' file
    NH.set(othersettings)
    NH.compute()

    Sigma8_value = NH.sigma8()
    print("",Sigma8_value,"")
    
    ideal_value = ((Sig_8)/(Sigma8_value))**2*(A_s)
    
    NH.struct_cleanup()
    
    return ideal_value

In [6]:
def make_Pk(spec_file,omega_h_h):
    
    othersettings = { 'ncdm_psd_filenames': spec_file,
                      'omega_cdm': measured_omegacdm - omega_h_h, 
                      'omega_ncdm': omega_h_h, 
                      'A_s': ideal_sigma8(spec_file, omega_h_h),
                    }
    
    for key, value in othersettings.items():
        print(key, ' : ', value)
    for key, value in commonsettings.items():
        print(key, ' : ', value)
    
    # array of k values in 1/Mpc


    NH = Class()
    NH.set(commonsettings)
    NH.set(othersettings)
    NH.compute()
    
    new_sig8 = NH.sigma8()
    pkNH = [] 
    
        #MPk- Matter Power Spectrum
    for k in kvec:
        pkNH.append(NH.pk(k,0.))
    h = NH.h()
    NH.struct_cleanup()
    

    np.save(spec_file+ '-Pknew',np.array(pkNH))
    np.save(spec_file+'-knew',kvec/h)
    
    return new_sig8

**With k = 20**

In [7]:
%%time
make_Pk(spec_file, omega_h_h)

 0.7866071721193716 
ncdm_psd_filenames  :  Spec
omega_cdm  :  0.05941682610033301
omega_ncdm  :  0.05938317389966699
A_s  :  2.2322621700046643e-09
N_ncdm  :  1
use_ncdm_psd_files  :  1
m_ncdm  :  7100
T_ncdm  :  0.7
output  :  mPk
P_k_max_1/Mpc  :  100.0
ncdm_fluid_approximation  :  3
background_verbose  :  1
Maximum q  :  100
CPU times: user 2min 17s, sys: 1.67 s, total: 2min 19s
Wall time: 2min 19s


0.8109999999999996

**With k = 4**

In [8]:
a,b,c,d,e = xy_values('1x0.00049x3e-09-data.npz', 4)

In [9]:
np.savetxt("Spec", np.column_stack((a,b)))
spec_file  = 'Spec'

In [10]:
%%time
make_Pk(spec_file, omega_h_h)

 0.7537938931744148 
ncdm_psd_filenames  :  Spec
omega_cdm  :  0.05941682610033301
omega_ncdm  :  0.05938317389966699
A_s  :  2.430836639371758e-09
N_ncdm  :  1
use_ncdm_psd_files  :  1
m_ncdm  :  7100
T_ncdm  :  0.7
output  :  mPk
P_k_max_1/Mpc  :  100.0
ncdm_fluid_approximation  :  3
background_verbose  :  1
Maximum q  :  100
CPU times: user 4min 33s, sys: 4.28 s, total: 4min 37s
Wall time: 4min 36s


0.8109999999999995

In [21]:
M0 = 3.2e12 # M_sol / h

k_vals = np.load('Spec-knew.npy')
Pk_vals = np.load('Spec-Pknew.npy')

P_spline  = sp.CubicSpline(k_vals,Pk_vals)

def R(M):
        c = 2.5
        G = 6.67e-11 # m^3 / kg / s^2
        H100 = 100 * (1000/Mpc) # km/s/Mpc -> 1/s
        omegah2 = 0.1188

        rhobar = omegah2 * 3 * H100**2 / (8 * np.pi * G) / M_sol * Mpc**3 # kg / m^3 -> M_sol / Mpc^3
        return (3 * M / (4 * np.pi * rhobar * c**3))**(1/3) # Mpc




def W(k,R):
        if np.isscalar(k):
            if k * R > 1:
                return 0
            else:
                return 1
        else:
            result = np.zeros(len(k))
            for i in range(len(k)):
                if k[i] * R < 1 and k[i+1] * R > 1:
                    result[i] = (1 - k[i] * R)/(k[i+1]*R - k[i]*R)
                elif k[i] * R < 1:
                    result[i] = 1
            return result



def S(M):
        Rv = R(M)
        
        integrand = k_vals**2 * Pk_vals * W(k_vals, Rv)**2 / (2 * ( np.pi**2))
        return trap(integrand,k_vals)

def R(M):
        c = 2.5
        G = 6.67e-11 # m^3 / kg / s^2
        H100 = 100 * (1000/Mpc) # km/s/Mpc -> 1/s
        omegah2 = 0.1188

        rhobar = omegah2 * 3 * H100**2 / (8 * np.pi * G) / M_sol * Mpc**3 # kg / m^3 -> M_sol / Mpc^3
        return (3 * M / (4 * np.pi * rhobar * c**3))**(1/3) # Mpc

    
    
    

def dNdlnM(M): ##10^8
    Rv = R(M)
    
    return 1 / 44.5 / (6 * np.pi**2) * (M0 / M) / Rv**3 / np.sqrt(2 * np.pi * ( S(M) - S(M0))) * P_spline(1/Rv)

Mass_values = np.logspace(0, 8)
print(Mass_values)


dnvalues = np.array(len(Mass_values))
for i in range(0, len(Mass_values)):
    
    dnvalues[i] = dNdlnM(Mass_values[i])
    

[1.00000000e+00 1.45634848e+00 2.12095089e+00 3.08884360e+00
 4.49843267e+00 6.55128557e+00 9.54095476e+00 1.38949549e+01
 2.02358965e+01 2.94705170e+01 4.29193426e+01 6.25055193e+01
 9.10298178e+01 1.32571137e+02 1.93069773e+02 2.81176870e+02
 4.09491506e+02 5.96362332e+02 8.68511374e+02 1.26485522e+03
 1.84206997e+03 2.68269580e+03 3.90693994e+03 5.68986603e+03
 8.28642773e+03 1.20679264e+04 1.75751062e+04 2.55954792e+04
 3.72759372e+04 5.42867544e+04 7.90604321e+04 1.15139540e+05
 1.67683294e+05 2.44205309e+05 3.55648031e+05 5.17947468e+05
 7.54312006e+05 1.09854114e+06 1.59985872e+06 2.32995181e+06
 3.39322177e+06 4.94171336e+06 7.19685673e+06 1.04811313e+07
 1.52641797e+07 2.22299648e+07 3.23745754e+07 4.71486636e+07
 6.86648845e+07 1.00000000e+08]


IndexError: index 100 is out of bounds for axis 0 with size 100